# Import necessary libraries

In [1]:
import numpy as np 
import pandas as pd 
import keras 
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import time
import statistics

# Experiments: Histology-MNIST (28x28)

## Experiment 1: PCA + AlexNet & PCA + LeNet 5

In [ ]:
# read dataset
histology_dataset = pd.read_csv("./hmnist_28_28_L.csv")

# process and formatting
X = histology_dataset.iloc[:,0:784]
y = histology_dataset.loc[:,'label']
X_tensor = X.values.reshape(5000, 28, 28, 1)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y, test_size=0.2, shuffle=True, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=11)

### PCA

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
pca = PCA(n_components=332)
start_time = time.time()
principalComponents = pca.fit_transform(X_scaled)
print("train time in seconds:", time.time() - start_time)
principalDf = pd.DataFrame(data = principalComponents)

# train test split
X_train, X_test, y_train, y_test = train_test_split(principalDf, y, test_size=0.2, shuffle=True, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=11)

### AlexNet fully connected layers

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
accuracy_pca_alex = []

for i in range(20):
  print(i)
  with tf.device('/device:GPU:0'):
    model = keras.models.Sequential([
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])
    adam = Adam(lr=5e-4)
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=25, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    start_time = time.time()
    model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[earlyStopping, mcp_save])
    print("train time in seconds:", time.time() - start_time)
    prediction = model.predict_classes(X_test)
    report = classification_report(y_test, prediction, output_dict=True)
    accuracy_pca_alex.append(report['accuracy'])

### Le-Net 5 fully connected layers

In [ ]:
accuracy_pca_lenet = []

for i in range(20):
  print(i)
  model = keras.models.Sequential([
      keras.layers.Dense(332, activation='tanh'), #C5
      keras.layers.Dense(84, activation='tanh'), #F6
      keras.layers.Dense(10, activation='softmax') #Output layer
  ])
  adam = Adam(lr=5e-4)
  earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='min')
  mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
  model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
  start_time = time.time()

  model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[earlyStopping, mcp_save])
  print("train time in seconds:", time.time() - start_time)
  prediction = model.predict_classes(X_test)
  report = classification_report(y_test, prediction, output_dict=True)
  accuracy_pca_lenet.append(report['accuracy'])

## Experiment 2: Full AlexNet

In [ ]:
# read dataset
histology_dataset = pd.read_csv("./hmnist_28_28_L.csv")

# process and formatting
X = histology_dataset.iloc[:,0:784]
y = histology_dataset.loc[:,'label']
X_tensor = X.values.reshape(5000, 28, 28, 1)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y, test_size=0.2, shuffle=True, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=11)

### Train AlexNet

In [ ]:
accuracy_alexnet = []

for i in range(20):
  print(i)
  with tf.device('/device:GPU:0'):
    model = keras.models.Sequential([
      keras.layers.UpSampling2D(size=(8,8)),
      keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(28,28,1)),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Flatten(),
      keras.layers.Dense(4096, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(4096, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(10, activation='softmax')
  ])
    adam = Adam(lr=5e-4)
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    X_train = tf.cast(X_train, tf.float32)
    X_test = tf.cast(X_test, tf.float32)
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[earlyStopping, mcp_save])
    print("train time in seconds:", time.time() - start_time)
    prediction = model.predict_classes(X_test)
    report = classification_report(y_test, prediction, output_dict=True)
    accuracy_alexnet.append(report['accuracy'])

## Experiment 3: Full Le-Net 5

In [ ]:
# read dataset
histology_dataset = pd.read_csv("./hmnist_28_28_L.csv")

# process and formatting
X = histology_dataset.iloc[:,0:784]
y = histology_dataset.loc[:,'label']
X_tensor = X.values.reshape(5000, 28, 28, 1)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y, test_size=0.2, shuffle=True, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=11)

### Train Le-Net 5

In [ ]:
# read dataset
histology_dataset = pd.read_csv("./hmnist_28_28_L.csv")

# process and formatting
X = histology_dataset.iloc[:,0:784]
y = histology_dataset.loc[:,'label']
X_tensor = X.values.reshape(5000, 28, 28, 1)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y, test_size=0.2, shuffle=True, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=11)

In [ ]:
accuracy_lenet = []

for i in range(20):
  print(i)
  model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=(28,28,1), padding='same'), #C1
    keras.layers.AveragePooling2D(), #S2
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'), #C3
    keras.layers.AveragePooling2D(), #S4
    keras.layers.Flatten(), #Flatten
    keras.layers.Dense(120, activation='tanh'), #C5
    keras.layers.Dense(84, activation='tanh'), #F6
    keras.layers.Dense(10, activation='softmax') #Output layer
])
  adam = Adam(lr=5e-4)
  earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='min')
  mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
  model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
  start_time = time.time()
  history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[earlyStopping, mcp_save])
  print("train time in seconds:", time.time() - start_time)
  prediction = model.predict_classes(X_test)
  report = classification_report(y_test, prediction, output_dict=True)
  accuracy_lenet.append(report['accuracy'])

## Experiment 4: TDAsweep + AlexNet & TDAsweep + Le-Net 5

### Load already processed dataset (TDAsweep)

In [ ]:
tdasweep_train = pd.read_csv("../prerun_tdasweep/histology_tdasweep")
tdasweep_test = pd.read_csv("../prerun_tdasweep/histology_tdasweep_test")

tdasweep_train_X = tdasweep_train.iloc[:,0:168]
tdasweep_train_y = tdasweep_train.loc[:,'labels']

tdasweep_test_X = tdasweep_test.iloc[:,0:168]
tdasweep_test_y = tdasweep_test.loc[:,'labels']

# train test split
tdasweep_train_X, tdasweep_val_X, tdasweep_train_y, tdasweep_val_y = train_test_split(tdasweep_train_X, tdasweep_train_y, test_size=0.1, shuffle=True, random_state=11)

### TDAsweep + AlexNet

In [ ]:
accuracy_tdasweep_alex = []

for i in range(20):
  print(i)
  with tf.device('/device:GPU:0'):
    model = keras.models.Sequential([
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])
    adam = Adam(lr=5e-4)
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=25, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    start_time = time.time()
    model.fit(tdasweep_train_X, tdasweep_train_y, epochs=50, validation_data=(tdasweep_val_X, tdasweep_val_y), callbacks=[earlyStopping, mcp_save])
    print("train time in seconds:", time.time() - start_time)
    prediction = model.predict_classes(tdasweep_test_X)
    report = classification_report(tdasweep_test_y, prediction, output_dict=True)
    accuracy_tdasweep_alex.append(report['accuracy'])

### TDAsweep + LeNet 5

In [ ]:
accuracy_tdasweep_lenet = []

for i in range(20):
  print(i)
  model = keras.models.Sequential([
      keras.layers.Dense(332, activation='tanh'), #C5
      keras.layers.Dense(84, activation='tanh'), #F6
      keras.layers.Dense(10, activation='softmax') #Output layer
  ])
  adam = Adam(lr=5e-4)
  earlyStopping = EarlyStopping(monitor='val_accuracy', patience=25, verbose=0, mode='min')
  mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
  model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
  start_time = time.time()
  model.fit(tdasweep_train_X, tdasweep_train_y, epochs=50, validation_data=(tdasweep_val_X, tdasweep_val_y), callbacks=[earlyStopping, mcp_save])
  print("train time in seconds:", time.time() - start_time)
  prediction = model.predict_classes(tdasweep_test_X)
  report = classification_report(tdasweep_test_y, prediction, output_dict=True)
  accuracy_tdasweep_lenet.append(report['accuracy'])

## Overall comparison and evaluation

### Basic stats for accuracy

In [ ]:
import statistics 
pd.DataFrame(accuracy_alexnet).describe()
pd.DataFrame(accuracy_lenet).describe()
pd.DataFrame(accuracy_pca_alex).describe()
pd.DataFrame(accuracy_pca_lenet).describe()
pd.DataFrame(accuracy_tdasweep_alex).describe()
pd.DataFrame(accuracy_tdasweep_lenet).describe()


### Box-plot accuracy visualizations (over 20 runs)

#### TDAsweep vs AlexNet

In [ ]:
accuracies = pd.DataFrame(np.vstack([np.asarray(accuracy_alexnet), np.asarray(accuracy_tdasweep_alex)]))

fig, ax = plt.subplots()
ax.boxplot(accuracies)
plt.xticks([1,2], ('AlexNet', 'TDAsweep + AlexNet (FC)'))

#### TDAsweep vs LeNet 5

In [ ]:
accuracies = pd.DataFrame(np.vstack([np.asarray(accuracy_lenet), np.asarray(accuracy_tdasweep_lenet)]))

fig, ax = plt.subplots()
ax.boxplot(accuracies)
plt.xticks([1,2], ('Le-Net 5', 'TDAsweep + Le-Net 5 (FC)'))

#### (Le-Net 5 fully connected) TDAsweep vs PCA

In [ ]:
accuracies = pd.DataFrame(np.vstack([np.asarray(accuracy_pca_lenet), np.asarray(accuracy_tdasweep_lenet)]))

fig, ax = plt.subplots()
ax.boxplot(accuracies)
plt.xticks([1,2], ('PCA + Le-Net 5 (FC)', 'TDAsweep + Le-Net 5 (FC)'))

#### (Le-Net 5 fully connected) TDAsweep vs PCA

In [ ]:
accuracies = pd.DataFrame(np.vstack([np.asarray(accuracy_pca_alex), np.asarray(accuracy_tdasweep_alex)]))

fig, ax = plt.subplots()
ax.boxplot(accuracies)
plt.xticks([1,2], ('PCA + AlexNet (FC)', 'TDAsweep + AlexNet (FC)'))

# Other Experiments

## OctMNIST

In [ ]:
tdasweep_train = pd.read_csv("../prerun_tdasweep/octmnist_tdasweep_train")
tdasweep_test = pd.read_csv("../prerun_tdasweep/octmnist_tdasweep_test")

tdasweep_train_X = tdasweep_train.iloc[:,0:332]
tdasweep_train_y = tdasweep_train.loc[:,'labels']

tdasweep_test_X = tdasweep_test.iloc[:,0:332]
tdasweep_test_y = tdasweep_test.loc[:,'labels']

# train test split
tdasweep_train_X, tdasweep_val_X, tdasweep_train_y, tdasweep_val_y = train_test_split(tdasweep_train_X, tdasweep_train_y, test_size=0.1, shuffle=True, random_state=11)

### TDAsweep + AlexNet

In [ ]:
accuracy_tdasweep = []

for i in range(20):
  print(i)
  with tf.device('/device:GPU:0'):
    model = keras.models.Sequential([
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])
    adam = Adam(lr=5e-4)
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=25, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    start_time = time.time()
    model.fit(tdasweep_train_X, tdasweep_train_y, epochs=50, validation_data=(tdasweep_val_X, tdasweep_val_y), callbacks=[earlyStopping, mcp_save])
    print("train time in seconds:", time.time() - start_time)
    prediction = model.predict_classes(tdasweep_test_X)
    report = classification_report(tdasweep_test_y, prediction, output_dict=True)
    accuracy_tdasweep.append(report['accuracy'])

## PneumoniaMNIST

In [ ]:
X_train = pd.read_csv("./pneumoniamnist_train_X.csv")
X_test = pd.read_csv("./pneumoniamnist_test_X.csv")
y_train = pd.read_csv("./pneumoniamnist_train_y.csv")
y_test = pd.read_csv("./pneumoniamnist_test_y.csv")

In [ ]:
X_train = X_train.iloc[:, 1:785]
X_test = X_test.iloc[:, 1:785]
y_train = y_train.iloc[:,1]
y_test = y_test.iloc[:,1]

X_train = X_train.values.reshape(4708, 28, 28, 1)
X_test = X_test.values.reshape(624, 28, 28, 1)

### Full AlexNet

In [ ]:
accuracy_alexnet = []

for i in range(20):
  print(i)
  with tf.device('/device:GPU:0'):
    model = keras.models.Sequential([
      keras.layers.UpSampling2D(size=(8,8)),
      keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(28,28,1)),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Flatten(),
      keras.layers.Dense(4096, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(4096, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(10, activation='softmax')
  ])
    adam = Adam(lr=5e-4)
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    X_train = tf.cast(X_train, tf.float32)
    X_test = tf.cast(X_test, tf.float32)
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[earlyStopping, mcp_save])
    print("train time in seconds:", time.time() - start_time)
    prediction = model.predict_classes(X_test)
    report = classification_report(y_test, prediction, output_dict=True)
    accuracy_alexnet.append(report['accuracy'])

### TDAsweep + AlexNet

In [ ]:
tdasweep_train = pd.read_csv("../prerun_tdasweep/pneumoniamnist_tdasweep_train")
tdasweep_test = pd.read_csv("../prerun_tdasweep/pneumoniamnist_tdasweep_test")

tdasweep_train_X = tdasweep_train.iloc[:,0:332]
tdasweep_train_y = tdasweep_train.loc[:,'labels']

tdasweep_test_X = tdasweep_test.iloc[:,0:332]
tdasweep_test_y = tdasweep_test.loc[:,'labels']

# train test split
tdasweep_train_X, tdasweep_val_X, tdasweep_train_y, tdasweep_val_y = train_test_split(tdasweep_train_X, tdasweep_train_y, test_size=0.1, shuffle=True, random_state=11)

In [ ]:
accuracy_tdasweep = []

for i in range(20):
  print(i)
  with tf.device('/device:GPU:0'):
    model = keras.models.Sequential([
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])
    adam = Adam(lr=5e-4)
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=25, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    start_time = time.time()
    model.fit(tdasweep_train_X, tdasweep_train_y, epochs=50, validation_data=(tdasweep_val_X, tdasweep_val_y), callbacks=[earlyStopping, mcp_save])
    print("train time in seconds:", time.time() - start_time)
    prediction = model.predict_classes(tdasweep_test_X)
    report = classification_report(tdasweep_test_y, prediction, output_dict=True)
    accuracy_tdasweep.append(report['accuracy'])

# Experiment: octMNIST

## Read datasets

In [ ]:
X_train = pd.read_csv("./octmnist_train_X.csv")
X_test = pd.read_csv("./octmnist_test_X.csv")
y_train = pd.read_csv("./octmnist_train_y.csv")
y_test = pd.read_csv("./octmnist_test_y.csv")

# format
X_train = X_train.iloc[:, 1:785]
X_test = X_test.iloc[:, 1:785]
y_train = y_train.iloc[:,1]
y_test = y_test.iloc[:,1]
X_train = X_train.values.reshape(97477, 28, 28, 1)
X_test = X_test.values.reshape(1000, 28, 28, 1)

## Train AlexNet

In [ ]:
accuracy_alexnet = []

for i in range(20):
  print(i)
  with tf.device('/device:GPU:0'):
    model = keras.models.Sequential([
      keras.layers.UpSampling2D(size=(8,8)),
      keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(28,28,1)),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
      keras.layers.Flatten(),
      keras.layers.Dense(4096, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(4096, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(10, activation='softmax')
  ])
    adam = Adam(lr=5e-4)
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    X_train = tf.cast(X_train, tf.float32)
    X_test = tf.cast(X_test, tf.float32)
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[earlyStopping, mcp_save])
    print("train time in seconds:", time.time() - start_time)
    prediction = model.predict_classes(X_test)
    report = classification_report(y_test, prediction, output_dict=True)
    accuracy_alexnet.append(report['accuracy'])